Data preparation

In [ ]:
#For Colab

from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/MyDrive/CV_3
!ls

Mounted at /content/gdrive
/content/gdrive/MyDrive/CV_3
CV_3_train.ipynb  files        model.py		    __pycache__  train.py
data.py		  kaggle.json  new_data		    README.md
dataset		  logs	       people_segmentation  results
eval.py		  metrics.py   predict.py	    test_images


In [ ]:
!pip install -r requirements.txt

In [ ]:
from data import *
from train import *
from metrics import dice_loss, dice_coef, iou, dice_loss_m, dice_coef_m

In [ ]:
# Prepare dataset for model

np.random.seed(42)

""" Load the dataset """
data_path = "/content/gdrive/MyDrive/CV_3/dataset"
(train_x, train_y), (test_x, test_y) = load_data(data_path, split = 0.1)

print(f"Train:\t {len(train_x)} - {len(train_y)}")
print(f"Test:\t {len(test_x)} - {len(test_y)}")

""" Create directories to save the augmented data """
create_dir("new_data/train/image/")
create_dir("new_data/train/mask/")
create_dir("new_data/test/image/")
create_dir("new_data/test/mask/")

""" Data augmentation """
augment_data(train_x, train_y, "new_data/train/", augment=False)
augment_data(test_x, test_y, "new_data/test/", augment=False)

Train:	 2354 - 2354
Test:	 261 - 261


100%|██████████| 261/261 [01:50<00:00,  2.37it/s]


Training

In [ ]:
# Set dataset_path and start training

""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory for storing files """
create_dir("files")

""" Hyperparameters """
batch_size = 4
lr = 1e-4
num_epochs = 100
model_path = os.path.join("files", "model.h5")
csv_path = os.path.join("files", "data.csv")

""" Dataset """
dataset_path = "/content/gdrive/MyDrive/CV_3/new_data"
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "test")

train_x, train_y = load_data(train_path)
train_x, train_y = shuffling(train_x, train_y)
valid_x, valid_y = load_data(valid_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

""" Model """
model = deeplabv3_plus((H, W, 3))
model.compile(loss=dice_loss_m, optimizer=Adam(lr), metrics=[dice_coef_m, iou, Recall(), Precision()])

callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=False),
    ]

model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        callbacks=callbacks
    )

Train: 11770 - 11770
Valid: 261 - 261
94781440/94765736 [==============================] - 1s 0us/step
Epoch 1/100
2943/2943 [==============================] - ETA: 0s - loss: 0.1645 - dice_coef_m: 0.8355 - iou: 1.1636 - recall: 0.4882 - precision: 0.9773
Epoch 1: val_loss improved from inf to 0.15001, saving model to files/model.h5
2943/2943 [==============================] - 3380s 1s/step - loss: 0.1645 - dice_coef_m: 0.8355 - iou: 1.1636 - recall: 0.4882 - precision: 0.9773 - val_loss: 0.1500 - val_dice_coef_m: 0.8500 - val_iou: 1.3426 - val_recall: 0.5550 - val_precision: 0.9992 - lr: 1.0000e-04
Epoch 2/100
2943/2943 [==============================] - ETA: 0s - loss: 0.1627 - dice_coef_m: 0.8373 - iou: 1.3189 - recall: 0.5296 - precision: 0.9984
Epoch 2: val_loss did not improve from 0.15001
2943/2943 [==============================] - 3328s 1s/step - loss: 0.1627 - dice_coef_m: 0.8373 - iou: 1.3189 - recall: 0.5296 - precision: 0.9984 - val_loss: 0.1500 - val_dice_coef_m: 0.8500 -

KeyboardInterrupt: ignored

Predicting

In [1]:
# Upload images to test_images/image and start predicting
!python predict.py
# Results are on test_images/mask folder

python3: can't open file 'predict.py': [Errno 2] No such file or directory
